## Partie 1
Implémenter la méthode des treillis pour le calcul des prix d’options (pour les différents types d’options). 
On testera la méthode sur différentes instances en faisant varier les paramètres (N : le nombre de périodes, r : taux de crédit, d et u). 
On essayera de considérer aussi des instances assez larges. 

In [4]:
import matplotlib.pyplot as plt
import numpy as np

In [5]:
def assetTreeValue(n, S, u, d):  
    asset = np.zeros((n+1,n+1))
    asset[0,0] = S
    for i in range(1,n+1):
        asset[i,0] = asset[i-1,0]*u
        for j in range(1,i+1):
            asset[i,j] = asset[i-1,j-1]*d
    return asset    

In [6]:
def calculationOptionPrice(n, assetValue, K, r, u, d, PutCall, optiontype):
    value = np.zeros((n+1,n+1))
    proba = (1+r-d)/(u-d)
    print('Probabilité Risque-neutre : ',proba)
    
    for j in range(n+1):
        if PutCall=="C": # Call
            payoff = max(0, (assetValue[n,j]-K))
            value[n,j] = payoff
        elif PutCall=="P": #Put
            print(j)
            payoff = max(0, (K-assetValue[n,j]))
            value[n,j] = payoff
    
    if optiontype == "USA":
         for i in range(n-1,-1,-1):
            for j in range(i+1):
                if PutCall=="P":
                    value[i,j] = max(0, K-assetValue[i,j], (1/1+r)*(proba*value[i+1,j]+(1-proba)*value[i+1,j+1]))
                elif PutCall=="C":
                    value[i,j] = max(0, assetValue[i,j]-K, (1/1+r)*(proba*value[i+1,j]+(1-proba)*value[i+1,j+1]))
    
    if optiontype == "EUR":
         for i in range(n-1,-1,-1):
            for j in range(i+1):
                value[i,j] = (1/1+r)*(proba*value[i+1,j]+(1-proba)*value[i+1,j+1])
                    
    return value

In [7]:
# test de visualisation
def drawAssetTree(n, asset):
    positionX = np.zeros((n+1,n+1))
    positionY = np.zeros((n+1,n+1))
    positionX[0][0] = 0.10
    positionY = 0.5
    
    plt.xlim(0,1) 
    plt.figtext(0.10,0.5,asset[0,0])
    
    for i in range(0,n+1):
        for j in range(i):
            plt.figtext(i-0.4,i+0.2,asset[i,j])
            plt.annotate('',xy=(i-0.4,i+0.2), xytext=(0.10,0.5), arrowprops=dict(facecolor='g',shrink=0.01))

    plt.axis('off')
    plt.show()

In [8]:
# test de visualisation
def drawTree(n, asset):
    plt.xlim(0,1) 
    plt.figtext(0.10,0.5,asset[0,0])
    
    plt.figtext(0.6,0.5+0.20,asset[1,0])
    plt.figtext(0.6,0.5-0.20,asset[1,1])
    
    plt.annotate('',xy=(0.6,0.5+0.25), xytext=(0.1,0.5), arrowprops=dict(facecolor='g',shrink=0.01))
    plt.annotate('',xy=(0.6,0.5-0.25), xytext=(0.1,0.5), arrowprops=dict(facecolor='g',shrink=0.01))
    
    plt.figtext(0.9,0.5+0.35,asset[2,0])
    plt.figtext(0.9,0.5,asset[2,1])
    plt.figtext(0.9,0.5-0.35,asset[2,2])
    
    plt.annotate('',xy=(0.9,0.5+0.40), xytext=(0.7,0.5+0.25), arrowprops=dict(facecolor='g',shrink=0.01))
    plt.annotate('',xy=(0.9,0.5), xytext=(0.7,0.5+0.25), arrowprops=dict(facecolor='g',shrink=0.01))
    
    plt.annotate('',xy=(0.9,0.5), xytext=(0.7,0.5-0.25), arrowprops=dict(facecolor='g',shrink=0.01))
    plt.annotate('',xy=(0.9,0.5-0.4), xytext=(0.7,0.5-0.25), arrowprops=dict(facecolor='g',shrink=0.01))
    
    plt.axis('off')
    plt.show()

In [307]:
asset = assetTreeValue(2, 100, 1.1, 0.9)
print(asset)
print(' ')
#drawTree(2, asset)
optionvalue = calculationOptionPrice(2, asset, 91, 0.05, 1.1, 0.9, "C", "EUR")

print(' ')
print(optionvalue[0][0])
#drawTree(2, optionvalue)

[[100.   0.   0.]
 [110.  90.   0.]
 [121.  99.  81.]]
 
Probabilité Risque-neutre :  0.7499999999999999
 
21.91218750000002


## Partie 2
Ecrire le modèle qui permet de minimiser la CVaR pour une probabilité donnée  (On donne le modèle général avec un nombre d’actifs n, un nombre de scénarios S, un vecteur rendement  et un rendement minimum R. 

<img src="pics\Partie 2.png" width="480" />

In [316]:
from scipy.optimize import minimize

In [324]:
alpha = 0.99 # Porba
S = 3 # Nombre de scénarios
R = 0.10

X = np.array([0, 0.50, 0.25, 0.25]) # Réparitition des actifs,  X[0] == gamma
b = np.array([0, 20, 30, 25]) # Prix des obligations, b[0]=0 pour la fonction objective
Y = np.array([0, 0.30, 0.30, 0.40]) # vecteur de probabilités
E = np.array([0, 0.1073, 0.07415, 0.0627]) # Espérence de rendement des actifs

In [325]:
# X[0] Gamma, X[1...N] Actifs risqués 
def func_minimise_ES(X):
    sum = 0
    for i in range(S):
        loose_function = 0
        for j in range (1, X.size):
            loose_function = loose_function + max(((b[j]-Y[i])*X[j]-X[0]),0)
            
    return X[0] + (1/(1-alpha)*S)*loose_function

In [326]:
# S fois cette contrainte
def constraint1(X,s):
    sum = 0
    for j in range (1,X.size):
        sum = sum + (b[j]-Y[s])*X[j]
    return max((sum-X[0]),0) - (sum-X[0]) 

In [327]:
# S fois cette contrainte
def constraint2(X,s):
    sum = 0
    for j in range (1,X.size):
        sum = sum + (b[j]-Y[s])*X[j]
    return max((sum-X[0]),0) 

In [331]:
def constraint3(X):
    sum = 0
    for i in range(1,X.size):
        sum = sum + X[i]
    return 1 - sum
    

In [332]:
def minimise_ES_constraint4(X):
    sum = 0
    for i in range(1,X.size):
        sum += X[i]*E[i]
    return sum - R 
    

In [333]:
# constraints
ineq_cons = ([{'type':'ineq','fun': lambda i : constraint1(X,i)} for i in range(S)] +
            [{'type':'ineq','fun': lambda i : constraint2(X,i)} for i in range(S)] +
            [{'type':'eq','fun': constraint3}, {'type':'ineq','fun': minimise_ES_constraint4}])

# bonds 
bond = (0, 1.0)
ineq_bond = [(None,None)] + [(0,1.0) for i in range(X.size-1)] # x1 >= 0, x2 >= 0, ...


# Ne fonctionne pas comme il le faut 
sol = minimize(func_minimise_ES,
               x0=X, 
               method='TNC',  # Il faut trouver la bonne méthode
               bounds=ineq_bond, 
               options={'disp': True},
               constraints=ineq_cons)
print(sol)

     fun: 0.024448063789509637
     jac: array([1.00000000e+00, 5.90882248e+03, 0.00000000e+00, 0.00000000e+00])
 message: 'Linear search failed'
    nfev: 380
     nit: 4
  status: 4
 success: False
       x: array([0.02444806, 0.00124102, 0.        , 0.        ])


## Partie 3 (à revoir)

Considérons le modèle de Markowitz maximisant le rendement du portefeuille et assurant une CVaR  ne dépassant pas un certain seuil.  
Ecrire le modèle correspondant et le résoudre pour certaines données aléatoires. 

<img src="pics\Partie 3.png" width="494" />

In [357]:
from scipy.optimize import linprog

In [347]:
# Définition des nouvelles varibales 

J = np.array([0, 0.50, 0.25, 0.25]) # Ensemble d'indices sur les différentes valeurs disponibles
U = np.array([0, 1, 3, 4 ]) # Risque maximal toléré pour une valeur Alpha j

In [348]:
# X[0] Gamma, X[1...N] Actifs risqués 
def func_maximise_yield(X):
    rendement = 0
    for i in range(1,X.size):
        rendement += X[i]*E[i]
    return rendement

In [349]:
# S fois cette contrainte
def maximise_yield_constraint1(X,alpha_j):
    sum = 0
    for i in range(S):
        loose_function = 0
        for j in range (1, X.size):
            loose_function = loose_function + max(((b[j]-Y[i])*X[j]-X[0]),0)
            
    return U[alpha_j] - (X[0] + (1/(1-alpha)*S)*loose_function) 

In [358]:
# constraints
ineq_cons = ([{'type':'ineq','fun': lambda i : maximise_yield_constraint1(X,i)} for i in range(J.size)] +
            [{'type':'ineq','fun': lambda i : constraint2(X,i)} for i in range(S)] +
            [{'type':'ineq','fun': lambda i : constraint1(X,i)} for i in range(S)] + 
             [{'type':'eq','fun': constraint3}])

# bonds 
bond = (0, 1.0)
ineq_bond = [(None,None)] + [(0,1.0) for i in range(X.size-1)] # x1 >= 0, x2 >= 0, ...
